# 强化学习（2）
上一篇介绍了强化学习的基本概念，并引入了状态值函数的定义，这一篇主要讲述估计状态值函数的三种方法：动态规划、蒙特卡洛、时间差分。
我们先回顾值函数的形式：
$$V^{\pi}(s) = E_\pi[\sum\limits_{k=0}^{\infty}\gamma^k r_{t+k+1}|s_t=s]$$

## 1.动态规划
利用贝尔曼方程，我们可以证明值函数具有递推性。
$$ \begin{aligned}V^\pi(s)&=E_\pi[\sum\limits_{k=0}^{\infty}\gamma^k r_{t+k+1}|s_t=s]\\&=\sum_{a}\pi(s,a)E_a\{r_{t+1}+\gamma \sum_{k=0}^\infty \gamma^k r_{t+k+2}|s_t=s\}\\&=\sum_{a}\pi(s,a) \sum_{s'}P(s,a,s')E\{r_{t+1}+\gamma \sum_{k=0}^\infty \gamma^k r_{t+k+2}|s_{t+1}=s'\}\\&=\sum_{a}\pi(s,a) \sum_{s'}P(s,a,s')\{R(s,a,s')+\gamma E_\pi[\sum_{k=0}^\infty \gamma^k r_{t+k+2}|s_{t+1}=s']\}\\&=\sum_{a}\pi(s,a) \sum_{s'}P(s,a,s')\{R(s,a,s')+\gamma V^\pi(s')\}\end{aligned}$$

上式中，$E_a[\cdot|s_t=s]$也可以写为$E[\cdot|s_t=s,a]$。

如果状态空间有限（假设为$|S|$)，我们可以写下$|S|$个方程组，每个方程有$|S|$个变量，求解这个方程组就可求得所有状态的值函数。

同样我们可以定义一个最优值函数（optimal value function）
$$ V^*(s)=\max_\pi V^\pi(s)$$
$V^*(s)$表示所有可能策略里能够得到的$V^\pi(s)$的最大值。最优值函数也有一个对应的贝尔曼方程版本：
$$ \begin{aligned}V^*(s)&=\max_a \sum_{s'\in S}P(s,a,s')\{R(s,a,s')+\gamma V^\pi(s')\}\\&=\sum_{s'\in S}P(s,a,s')\{R(s,a,s')+\gamma \max_a V^\pi(s')\}\\&=\sum_{s'\in S}P(s,a,s')\{R(s,a,s')+\gamma V^*(s')\}\end{aligned}$$
我们可以根据估计出的最优值函数制定最优的策略（由于$R(s)$是常数，可以丢掉）
$$\label{policy}\begin{equation}\pi^*(s)=\arg\max\limits_{a\in A}\sum_{s'\in S}P_{sa}(s')V^*(s')\end{equation}$$

$\pi^*$满足如下性质：

$$V^*(s)=V^{\pi^*}(s)\geq V^\pi(s)$$

注意到$\pi^*(s)$是所有状态下最优的策略，也就是说不论从以哪个状态做起始，都能按照它执行。

#### 值迭代（value iteration）
前面介绍了一种解方程组求解$V^\pi(s)$的方法，如果用克拉默法则，求解时间为$O(|S|!)$；如果用高斯消元法时间复杂度为$O(|S|^3)$；目前最快的算法时间复杂度为$O(|S|^{2.376})$。当状态空间较大时，求解线性方程组效率较低。

这里介绍另一种高效的算法：值迭代。该算法的步骤如下：
1. 对每个状态$s$，初始化$V(s):=0$
2. 重复直到收敛 {
    对每个状态，更新： $V(s)=R(s)+\max\limits_{a\in A} \sum\limits_{s'\in S}P_{sa}(s')V(s')$
}

更新策略有两种：
* 同步更新：一次性计算所有状态的新值，再覆盖旧值
* 异步更新：循环过程中一次更新一个状态的值

可以证明无论是同步更新还是异步更新，$V(s)$最后都会收敛于$V^*(s)$（具体怎么分析的，我还需要看更多文献，这里只是照搬结论）。有了收敛后的$V(s)$，我们可以用$\eqref{policy}$制定策略
时间复杂度分析：
设收敛时的迭代次数为$T$，那么值迭代的时间复杂度为$O(T|S|^2|A|)$。在一些场合下比如机器人寻路，可执行的动作只有前后左右四种，动作空间远低于状态空间，这种情况下值迭代远比高斯消元法高效。

#### 策略迭代（policy iteration）
前面介绍的值迭代是基于更新值函数得到策略的，这里介绍一种更新策略的方法——策略迭代。
策略迭代算法步骤如下：
1. 随机初始化$\pi$
2. 重复直至收敛 {
(a) 令$V:=V^\pi$
(b) 对每个状态$s$，计算$\pi(s):=\arg\max\limits_{a\in A}\sum\limits_{s'}P_{sa}(s')V(s')$
}

循环中我们首先为当前的策略计算一个值函数，接着再用这个值函数更新策略。注意到(a)步骤中我们可以通过求解一个$|S|$元线性方程组得到值函数。同样可以证明，经过有限次循环，策略迭代算法能够收敛。
时间复杂度分析：
一般情况下，策略迭代可以收敛于多项式时间$O(T(|S|^3+|S|^2|A|))$。考虑到所有的策略有$|A|^{|S|}$种可能，故最坏情况下策略迭代要指数时间才能收敛$O(|A|^{|S|})$。

#### 值迭代和策略迭代的对比
值迭代的输出是每个状态的最优值函数，再用这个最优值函数生成策略函数（时间复杂度为$O(|S||A|)$）；策略迭代则同时得到最优值函数和最优策略。
当状态空间$|S|$比较大，动作空间$|A|$较小时，策略函数要求解一个线性方程组的代价太高，因此我们倾向于使用值迭代。当动作空间$|A|$较大，状态空间$|S|$比较小时，策略迭代求解线性方程组十分快速，可以很快收敛。
考虑到实际中的问题大多都是状态比较多的，因此比较常用的是值迭代算法。

## 2.蒙特卡洛
蒙特卡洛也称为统计模拟方法，是一种用随机实验解决数值计算问题的方法。蒙特卡洛方法提出于20世纪40年代的二战时期，被老美用来计算原子弹的弹道。数学家冯·诺伊曼用驰名世界的赌城—摩纳哥的Monte Carlo—来命名这种方法，为它蒙上了一层神秘色彩。在这之前蒙特卡洛方法就已经存在。1777年，法国数学家布丰（Georges Louis Leclere de Buffon，1707—1788）提出用投针实验的方法求圆周率$\pi$。这被认为是蒙特卡洛方法的起源。


给定某个策略$\pi$，蒙特卡洛估计每个状态$V^\pi(s_i),(i=1,2,...,|S|)$的步骤是这样的：
1. 从状态集随机挑选一个起始状态
2. 执行策略$\pi$，生成一个episode。每执行一次策略，我们可以获得执行路径上每个状态的累积回报样本。
3. 我们运行系统足够多次，为每个状态获得足够多的值函数样本，将这些样本平均，就得到其值函数的估计。

根据平均方式的不同，MC方法可以分为：First-visit MC和Every-visit MC。
#### First-visit MC
用第一次访问$s$后的累积回报样本估计$V^\pi(s)$
![1](http://7xikew.com1.z0.glb.clouddn.com/53dd5510-ee17-47aa-91a4-25ca761272ad.png)

#### Every-visit MC
用所有访问过s后的累积回报样本估计$V^\pi(s)$。算法流程如下（原论文没给出，自己P了一张）：
![2](http://7xikew.com1.z0.glb.clouddn.com/75fb1e20-0084-4886-b555-be49f97dfeaf.jpg)

可以保证，当运行次数趋于无限，这个估计就会逐渐收敛于$V^\pi(s)$。
前面介绍了如何用MC估计一个策略的值函数，那么如何用估计到的值函数求解最优策略？
下图为广义策略迭代示意图，evaluation阶段，我们不断修正值函数，使其逼近当前策略下真实的值函数。improvement阶段，我们用估计到的值函数贪心地更新策略。
![3](http://7xikew.com1.z0.glb.clouddn.com/f8af90ec-1a92-4c68-87ff-9a43120fcc63.png)
很容易想到，我们用MC做evaluation估计值函数，再根据估计的值函数用贪心算法更新当前策略不就行了？如下图所示，E表示evaluation，I表示更新策略。
![4](http://7xikew.com1.z0.glb.clouddn.com/9b08bd08-5dd2-4ec3-a3ec-660f2d47d746.png)

## 3. 时间差分
动态规划算法有如下特性：
* 需要环境模型，即状态转移概率$P_{sa}$
* 状态值函数的估计是自举的(bootstrapping)，即当前状态值函数的更新依赖于已知的其他状态值函数。

蒙特卡罗方法的特点则有：
* 不需要环境模型
* 状态值函数的估计是相互独立的
* 只能用于episode tasks

时间差分是蒙特卡洛和动态规划的结合，因此具有两者的优点：1）不需要环境知识，只需要根据经验求得最优策略。 2）不局限与episode task，可以用于连续任务



#### constant-$\alpha$ MC
这里介绍MC的一个变种。它的估计式是：
$$V(s_t) \leftarrow V(s_t) + \alpha[R_t - V(s_t)] \tag {1}$$

其中$R_t=\sum\limits_{k=0}^\infty \gamma^k R(s_{t+k})$是$t$时刻后的累积回报（可以认为是对$V(s_t)$的估计），$\alpha$是步长，$R_t-V(s_t)$是一个增量，用于修正$V(s_t)$。constant-$\alpha$ MC方法的缺点是需要等到整个episode结束才能计算$R_t$，以更新$V(s_t)$。
这里有必要对$V(s_t)$、$V^\pi(s)$和$V^\pi(s_t)$做个区别。
首先$V(s_t)$不是期望，只是一个sample。$V^\pi(s)$和$V^\pi(s_t)$都是数学期望。
* $V(s_t)$是$t$时刻的状态值函数，是对$V^\pi(s_t)$的估计值，$s_t$还没有给定，可以是任意状态
* $V^\pi(s_t)$是策略$\pi$下状态$s_t$的累积回报的期望，但$s_t$没有给定
* $V^\pi(s)$是策略$\pi$下状态$s_t$的累积回报期望，给定了$s_t=s$，产生了立即回报$R(s)$。
#### Temporal difference
constant-$\alpha$ MC由于要等到一段episode结束才能计算所有的累积回报，因此无法推广到continuing task。与MC方法不同的是，TD(0)方法只需要下一时刻的信息来更新$V(s_t)$：
$$ V(s_t)\leftarrow V(s_t)+\alpha[R(s_{t})+\gamma V(s_{t+1})-V(s_t)]$$
其中，$R(s_{t})$是t时刻的立即回报，$V_{s_{t+1}}$是下个时刻累积回报的估计。从式中可以看到，TD(0)的更新过程具有和DP类似的自举性质。
TD(0)的算法过程如下图所示：
![5](http://7xikew.com1.z0.glb.clouddn.com/baed76eb-588f-462b-8ee6-a0629b77a0d5.png)
用类似上节MC的手段，我们将TD(0)算法与policy improvement结合就能实现一个智能控制问题。

#### DP，MC，TD三者对比
考察公式
$$\begin{aligned}V^\pi(s)&=E_\pi[R_t|s_t=s]=E_\pi[\sum_{k=0}^\infty \gamma^k R(s_{t+k})|s_t=s]\\&=R(s)+\gamma E_\pi[\sum_{k=0}^\infty \gamma^k R(s_{t+k+1})|s_t=s]\\&=R(s)+\gamma \sum_{s'\in S}P_{s\pi(s)}(s')E_\pi[\sum_{k=0}^\infty \gamma^k R(s_{t+k+1})|s_{t+1}=s']\\&=R(s)+\gamma \sum_{s'\in S}P_{s\pi(s)}(s')V^\pi(s')\end{aligned}$$
可以发现：
* MC方法是以$E_\pi[R_t|s_t=s]$为目标进行估计
* DP方法是以$R(s)+\gamma \sum_{s'\in S}P_{s\pi(s)}(s')V^\pi(s')$为目标进行估计
* TD方法目标和DP的一样，只不过不是直接进行估计，而是用类似MC的方法通过采样去估计的。具体地，TD用$V(s')$替换$V^\pi(s')$，用$R(s)+\gamma V(s')$作为$R(s)+\gamma \sum_{s'\in S}P_{s\pi(s)}(s')V^\pi(s')$的估计



## 结语
到此，我们介绍了三种求解值函数的方法。文献1的公式有些看不懂，不过还是尝试着自己理解了，写下了自己对这些方法的理解。下次的内容是SARSA，Q-learning。

参考：
1.Reinforcement Learning: An Introduction
2.CS229-lecture-notes12
3.Littman M L, Dean T L, Kaelbling L P. On the Complexity of Solving Markov Decision Problems[C]// Eleventh International Conference on Uncertainty in Artificial Intelligence. 2013:394--402.
4.[增强学习（四） ----- 蒙特卡罗方法(Monte Carlo Methods) ](http://blog.csdn.net/zz_1215/article/details/44138881)
5.[增强学习（五）----- 时间差分学习(Q learning, Sarsa learning)](http://www.cnblogs.com/jinxulin/p/5116332.html?utm_source=tuicool&utm_medium=referral)